# Assignment 1

Lihua Xiong, lx559

## Part I: Plotting data with FIPS codes

** First, load and process data.**

In [1]:
import pandas as pd

In [2]:
# Note: read in as str.
data = pd.read_csv('national_county.txt', sep=",", header=None, dtype=str)
data.columns = ["STATE", "STATEFP", "COUNTYFP", "COUNTYNAME","CLASSFP"]

**Note:** 
If the FIPS is not converted to int, the following graph would be mismatched with the dataset.


In [3]:
data['FIPS'] = pd.to_numeric(data['STATEFP']+data['COUNTYFP'],\
                             downcast='integer')

In [4]:
dummy = pd.get_dummies(data['STATE'])
data['DUMMY'] = dummy['NY']

In [5]:
data.head()

,STATE,STATEFP,COUNTYFP,COUNTYNAME,CLASSFP,FIPS,DUMMY
0,AL,01,001,Autauga County,H1,1001,0
1,AL,01,003,Baldwin County,H1,1003,0
2,AL,01,005,Barbour County,H1,1005,0
3,AL,01,007,Bibb County,H1,1007,0
4,AL,01,009,Blount County,H1,1009,0


** Second, instantiate a US county map and visualize the data above.**

In [6]:
from bqplot import *
from bqplot import pyplot as plt

In [7]:
map_fig = plt.figure(title='Visulisation NY using State-County FIPS data',\
                     min_width=1300, min_height=800)
map_tt = Tooltip(fields=['name','color'], labels=['County Name', 'DUMMY'])

# Note: OrdinalColorScale is used for dummy.
map_res = plt.geo(map_data=topo_load('map_data/USCountiesMap.json'), stroke_color='black',\
                  colors={'default_color': 'Gray'},\
                  scales={'projection': AlbersUSA(),\
                         'color': OrdinalColorScale(domain=['Non-NY', 'NY'], colors=['#E91D0E','#2aa1ec'])},\
                  color=dict(zip(data['FIPS'], data['DUMMY'])), tooltip=map_tt)
map_fig

## Part II: Analyzing the HMDA dataset at county level

**Q1:** How many loans in each 'action_taken_name' category are there?

In [12]:
# Only read the first two cols for efficiency.
HMDA = pd.read_csv('hmda_lar-2012.csv', dtype=object, usecols=[0,1])

In [13]:
HMDA.head()

,action_taken,action_taken_name
0,1,Loan originated
1,1,Loan originated
2,6,Loan purchased by the institution
3,6,Loan purchased by the institution
4,1,Loan originated


In [22]:
result_1 = HMDA.groupby('action_taken_name')['action_taken'].count().to_frame()
result_1.columns = ['num_of_loans']
result_1

,num_of_loans
action_taken_name,
Application approved but not accepted,710706
Application denied by financial institution,2761964
Application withdrawn by applicant,1529647
File closed for incompleteness,524369
Loan originated,9783966
Loan purchased by the institution,3167577
Preapproval request approved but not accepted,64589
Preapproval request denied by financial institution,148733


**Q2:** If   you   group   by   ‘agency_abbr’,   how   many   loans   in   each   ‘action_taken_name’   category
are   there?   Construct   a   dataframe   indexed   by   ‘agency_abbr’   and   ‘action_taken_name’ and   save   it   into   a   ‘agency_action_stats.pkl’   file

In [23]:
# Only read the first four cols for efficiency.
HMDA = pd.read_csv('hmda_lar-2012.csv', dtype=object, usecols=[0,1,3])
HMDA.head()

,action_taken,action_taken_name,agency_abbr
0,1,Loan originated,HUD
1,1,Loan originated,NCUA
2,6,Loan purchased by the institution,CFPB
3,6,Loan purchased by the institution,CFPB
4,1,Loan originated,FDIC


In [24]:
result_2 = HMDA.groupby(['agency_abbr',\
                         pd.Grouper(key='action_taken_name')])\
['action_taken'].count().to_frame()
result_2.columns = ['num_of_loans']
result_2.head(10)

num_of_loans
agency_abbr action_taken_name                                               
CFPB        Application approved but not accepted                     307922
            Application denied by financial institution              1504528
            Application withdrawn by applicant                        482741
            File closed for incompleteness                            239245
            Loan originated                                          4106978
            Loan purchased by the institution                        2652899
            Preapproval request approved but not accepted              38915
            Preapproval request denied by financial institu...         91160
FDIC        Application approved but not accepted                      49036
            Application denied by financial institution               172266

In [25]:
result_2.to_pickle('agency_action_stats.pkl')

In [26]:
# The index satisfies the requirement above.
result_2.index

MultiIndex(levels=[['CFPB', 'FDIC', 'FRS', 'HUD', 'NCUA', 'OCC'], ['Application approved but not accepted', 'Application denied by financial institution', 'Application withdrawn by applicant', 'File closed for incompleteness', 'Loan originated', 'Loan purchased by the institution', 'Preapproval request approved but not accepted', 'Preapproval request denied by financial institution']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5], [0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7]],
           names=['agency_abbr', 'action_taken_name'])

**Q3:** Construct   a   dataframe   named   df_statecnty_stats   indexed   by   the   ‘msamd’   (which   is   the   same   as the   FIPS   state-county   code)   and   containing   the   following   columns:
- median_hud_inc   =   county   median   of   ‘hud_median_family_income’
- median_inc   =   median   county   income   of   loan   applicants
- mean_inc   =   mean   county   income   of   loan   applicants
- std_inc   =   std   county   income   of   loan   applicants
- mean_loan_to_inc   =   county   mean   loan   value   to   annual   income
- ‘mean_income_over_hud’   =   county   mean   of   applicant   income   /
hud_median_family_income
- ‘pct_denied_apps’   =   percentage   of   denied   loan   applications
- ‘pct_rentals’   =   can   you   compute   the   percent   rental   units   in   each   county?
- ‘pct_minority’   =   percent   minority   population
- ‘ln_pop’   =   log   of   the   county   population

**Step 1:** 
- Initiate an empty dataframe df_statecnty_stats.
- Construct FIPS using state_code and county_code.

**Note:** 
- The 'msamd' code provided by the original data has lots of missing values. Also it's not equivalent to FIPS. Thus FIPS is first generated via state_code and county_code, then used as index.

In [65]:
df_statecnty_stats = pd.DataFrame()

In [69]:
HMDA = pd.read_csv('hmda_lar-2012.csv', dtype=object, \
                   usecols=['state_code','county_code'])
FIPS = pd.to_numeric(HMDA['state_code'].str.zfill(2)+\
                     HMDA['county_code'].str.zfill(3),\
                     downcast='integer')

**Step 2:** Calculate median_hud_inc.

**Note:** 
- The 'msamd' code provided by the original data has lots of missing values. Also it's not equivalent to FIPS. Thus FIPS is first generated via state_code and county_code, then used as index.
- For efficiency, only the columns in use are read into the HMDA dataframe.
- I also tried reading the large csv file chunk by chunk and push it into HDF5 file (with 'write' mode). This process is also slow.

In [208]:
HMDA = pd.read_csv('hmda_lar-2012.csv', dtype=float, \
                   usecols=['hud_median_family_income'])
HMDA['FIPS'] = FIPS

In [71]:
df_statecnty_stats = HMDA.groupby('FIPS')['hud_median_family_income'].median().to_frame()
df_statecnty_stats.head()

,hud_median_family_income
FIPS,
1001.0,60100.0
1003.0,47600.0
1005.0,47600.0
1007.0,62800.0
1009.0,62800.0


**Step 3:** Calculate median_inc, mean_inc, std_inc.

**Note:** FIPS is used as index.

In [209]:
HMDA = pd.read_csv('hmda_lar-2012.csv', dtype=float, \
                   usecols=['applicant_income_000s'])
HMDA['FIPS'] = FIPS

In [210]:
df_statecnty_stats['median_inc'] = \
HMDA.groupby('FIPS')['applicant_income_000s'].median().to_frame()

In [77]:
df_statecnty_stats['mean_inc'] = \
HMDA.groupby('FIPS')['applicant_income_000s'].mean().to_frame()*1000

In [78]:
df_statecnty_stats['std_inc'] = \
HMDA.groupby('FIPS')['applicant_income_000s'].std().to_frame()

In [79]:
df_statecnty_stats.head()

,hud_median_family_income,median_inc,mean_inc,std_inc
FIPS,,,,
1001.0,60100.0,62.0,73.812070,51.222430
1003.0,47600.0,75.0,115.401952,170.518287
1005.0,47600.0,54.0,67.761468,53.990961
1007.0,62800.0,51.0,58.196023,34.825044
1009.0,62800.0,52.0,63.651608,54.360743


**Step 4:** Calculate mean_loan_to_inc, mean_income_over_hud.

In [80]:
HMDA = pd.read_csv('hmda_lar-2012.csv', dtype=float, \
                   usecols=['loan_amount_000s','applicant_income_000s','hud_median_family_income'])
HMDA['FIPS'] = FIPS

In [81]:
HMDA['loan_to_inc'] = HMDA['loan_amount_000s'].divide(HMDA['applicant_income_000s'])
HMDA['income_over_hud'] = HMDA['applicant_income_000s'].divide(HMDA['hud_median_family_income'])*1000.0

In [82]:
df_statecnty_stats['mean_loan_to_inc']=\
HMDA.groupby('FIPS')['loan_to_inc'].mean().to_frame()

In [83]:
df_statecnty_stats['mean_income_over_hud']=\
HMDA.groupby('FIPS')['income_over_hud'].mean().to_frame()

In [84]:
df_statecnty_stats.head()

,hud_median_family_income,median_inc,mean_inc,std_inc,mean_loan_to_inc,mean_income_over_hud
FIPS,,,,,,
1001.0,60100.0,62.0,73.812070,51.222430,2.203546,1.228430
1003.0,47600.0,75.0,115.401952,170.518287,2.285931,2.447498
1005.0,47600.0,54.0,67.761468,53.990961,2.043891,1.478975
1007.0,62800.0,51.0,58.196023,34.825044,1.965309,0.926688
1009.0,62800.0,52.0,63.651608,54.360743,2.094035,1.014303


**Step 5:** Calculate pct_denied_apps.

In [85]:
HMDA = pd.read_csv('hmda_lar-2012.csv', dtype=object, \
                   usecols=['action_taken','action_taken_name'])
HMDA['FIPS'] = FIPS

Create DUMMY for denied applicants.

In [86]:
dummy = pd.get_dummies(HMDA['action_taken_name'])
HMDA['DUMMY_denied'] = dummy['Application denied by financial institution']

In [87]:
df_statecnty_stats['pct_denied_apps']=\
HMDA.groupby('FIPS')['DUMMY_denied'].sum().divide(HMDA.groupby('FIPS')['DUMMY_denied'].count()).to_frame()

In [88]:
df_statecnty_stats.head()

,hud_median_family_income,median_inc,mean_inc,std_inc,mean_loan_to_inc,mean_income_over_hud,pct_denied_apps
FIPS,,,,,,,
1001.0,60100.0,62.0,73.812070,51.222430,2.203546,1.228430,0.160507
1003.0,47600.0,75.0,115.401952,170.518287,2.285931,2.447498,0.172720
1005.0,47600.0,54.0,67.761468,53.990961,2.043891,1.478975,0.280066
1007.0,62800.0,51.0,58.196023,34.825044,1.965309,0.926688,0.293742
1009.0,62800.0,52.0,63.651608,54.360743,2.094035,1.014303,0.199390


**Step 6:** Calculate pct_rentals.

In [90]:
HMDA = pd.read_csv('hmda_lar-2012.csv', dtype=object, \
                   usecols=['owner_occupancy_name'])
HMDA['FIPS'] = FIPS

In [93]:
dummy = pd.get_dummies(HMDA['owner_occupancy_name'])
HMDA['DUMMY_rentals'] = dummy['Not owner-occupied as a principal dwelling']

In [94]:
df_statecnty_stats['pct_rentals']=\
HMDA.groupby('FIPS')['DUMMY_rentals'].sum().divide(HMDA.groupby('FIPS')['DUMMY_rentals'].count()).to_frame()

In [95]:
df_statecnty_stats.head()

,hud_median_family_income,median_inc,mean_inc,std_inc,mean_loan_to_inc,mean_income_over_hud,pct_denied_apps,pct_rentals
FIPS,,,,,,,,
1001.0,60100.0,62.0,73.812070,51.222430,2.203546,1.228430,0.160507,0.078141
1003.0,47600.0,75.0,115.401952,170.518287,2.285931,2.447498,0.172720,0.235927
1005.0,47600.0,54.0,67.761468,53.990961,2.043891,1.478975,0.280066,0.095552
1007.0,62800.0,51.0,58.196023,34.825044,1.965309,0.926688,0.293742,0.045977
1009.0,62800.0,52.0,63.651608,54.360743,2.094035,1.014303,0.199390,0.049169


**Step 7:** Calculate pct_minority, ln_pop.

In [107]:
HMDA = pd.read_csv('hmda_lar-2012.csv', dtype=float, \
                   usecols=['minority_population','population', 'census_tract_number'])
HMDA['FIPS'] = FIPS

**Note:**
- The 'census_tract_number' column is the number of the census tract for the property. This code is only unique when combined with the state and county codes.
- The 'population' column is the total population in the tract.
- The 'minority_population' column is the percentage of minority population to total population for the census tract, carried to two decimal places.

**Thus,**
- county population = sum(popoulation of unique(census tract of the county))
- percent minority population of the county = sum over census tract(pct of minority of census tract * population of census tract)/county population

First we get the popoulation of each census tract.
- Note that the popuplation and FIPS+census_tract_number are one-on-one.
(check .std() == 0)
- Thus we could directly use .mean() after groupby.

Calculate census_minority = round(minority_population * census_population / 100).

In [168]:
census_minority = HMDA.groupby(['FIPS','census_tract_number'])['minority_population','population'].mean()

In [169]:
census_minority['min_pop'] = census_minority['minority_population'].multiply(census_minority['population']).\
divide(100.0).round().to_frame()

In [170]:
census_minority.head()

minority_population  population  min_pop
FIPS   census_tract_number                                          
1001.0 201.0                          16.270000      1912.0    311.0
       202.0                          61.110001      2170.0   1326.0
       203.0                          24.760000      3373.0    835.0
       204.0                           8.120000      4386.0    356.0
       205.0                          21.620001     10766.0   2328.0

Aggregate to county level.

In [175]:
county_data = census_minority.groupby(level=0)['population','min_pop'].sum()
county_data.head()

,population,min_pop
FIPS,,
1001.0,54571.0,12417.0
1003.0,182265.0,30065.0
1005.0,27457.0,14620.0
1007.0,22915.0,5724.0
1009.0,57322.0,6370.0


In [176]:
df_statecnty_stats['pct_minority'] = county_data['min_pop'].divide(county_data['population'])

In [177]:
df_statecnty_stats['ln_pop'] = np.log(county_data['population'])

In [178]:
df_statecnty_stats.head()

,hud_median_family_income,median_inc,mean_inc,std_inc,mean_loan_to_inc,mean_income_over_hud,pct_denied_apps,pct_rentals,ln_pop,pct_minority
FIPS,,,,,,,,,,
1001.0,60100.0,62.0,73.812070,51.222430,2.203546,1.228430,0.160507,0.078141,10.907258,0.227538
1003.0,47600.0,75.0,115.401952,170.518287,2.285931,2.447498,0.172720,0.235927,12.113217,0.164952
1005.0,47600.0,54.0,67.761468,53.990961,2.043891,1.478975,0.280066,0.095552,10.220376,0.532469
1007.0,62800.0,51.0,58.196023,34.825044,1.965309,0.926688,0.293742,0.045977,10.039547,0.249793
1009.0,62800.0,52.0,63.651608,54.360743,2.094035,1.014303,0.199390,0.049169,10.956440,0.111127


In [244]:
df_statecnty_stats.to_pickle('df_statecnty_stats.pkl')

**Step 8:** Plot mean_inc on a US county map as in Part 1.

In [245]:
map_fig = plt.figure(title='Visulisation of mean_inc on US County Map',\
                     min_width=1300, min_height=800)
map_tt = Tooltip(fields=['name','color'], labels=['County Name', 'mean_inc_000s'])

map_res = plt.geo(map_data=topo_load('map_data/USCountiesMap.json'), stroke_color='black',\
                  colors={'default_color': 'Black'},\
                  scales={'projection': AlbersUSA(),\
                         'color': ColorScale(colors=['Red','Gray','DeepSkyBlue'])},\
                  color=dict(zip(df_statecnty_stats.index.map(int), df_statecnty_stats['mean_inc'])), tooltip=map_tt)
map_fig

**Step 9 (Extra Credit):**  Can   you   have   the   user   select   which   column   to   plot   on   the   county   map   via
a   Dropdown   ipywidget?

In [289]:
from ipywidgets import interact

In [242]:
cols = list(df_statecnty_stats.columns.values)

In [290]:
# First we define a figure
def map_plot(x):
    
    map_fig = plt.figure(title='Visulisation of '+x+' on US County Map',\
                         min_width=1300, min_height=800)

    map_tt = Tooltip(fields=['name','color'], labels=['County Name', x])

    map_res = plt.geo(map_data=topo_load('map_data/USCountiesMap.json'), stroke_color='black',\
                      colors={'default_color': 'Black'},\
                      scales={'projection': AlbersUSA(),\
                             'color': ColorScale(colors=['Red','Gray','DeepSkyBlue'])},\
                      color=dict(zip(df_statecnty_stats.index.map(int), df_statecnty_stats[x])), tooltip=map_tt)
    return map_fig

In [292]:
interact(map_plot, x=cols)

<function __main__.map_plot>

**Note:** 
- Here I used the interact() function, which creates a Dropdown widget directly. 
- However, on my PC, when we select different colname, the plot doesn't update itself but displays a new one. Similar to this question: https://github.com/jupyter-widgets/ipywidgets/issues/1179, there might be some incompatibility issue between bqplot.pyplot.figure and ipywidgets, or we need another implementation.

## Appendix: 

**Read in the csv file chunk by chunk and push it to HDF5 table.**

from pandas import HDFStore
hdf = HDFStore('storage.h5', mode='w')

filename = 'hmda_lar-2012.csv'
reader = pd.read_csv(filename, chunksize=10000, dtype=object)
for chunk in reader:
    chunk['FIPS'] = pd.to_numeric(chunk['state_code'].head(3).str.zfill(2)+\
                                  chunk['county_code'].head(3).str.zfill(3),\
                                  downcast='integer')
                                  
    hdf.append('HMDA', chunk, format='table', data_columns=['FIPS'])

hdf.close()

**Note:** This implementation is not adopted because it's too slow.